In [0]:
%run /Workspace/setup_serverless

In [0]:
import sys

#sys.path.append('/Workspace/Users/erubio@yipitdata.com/freeport_service/')

sys.path.append("/Workspace/Repos/ETL_Production/freeport_service/")

from freeport_databricks.client.v1 import (
    get_or_create_deliverable,
    materialize_deliverable,
    release_materialization,
    list_deliverable_materializations,
    get_or_create_query_template,
    get_or_create_deliverable_group,
    sql_from_query_template
    
)

from freeport_databricks.client.api_client import (
    get,
    FREEPORT_DOMAIN
)
import json
import time

FREEPORT_DOMAIN

In [0]:
%sql
describe table yd_production.data_engineering_sandbox.variant_coltype_test_deliverable

if model version is expected to have schema changes keep an eye to set `allow_major_version = True` and `allow_minor_version = True`

In [0]:
deliverable = get_or_create_deliverable(
    title="variant_column_test_deliverable2",
    input_tables="yd_production.data_engineering_sandbox.variant_coltype_test_deliverable",
    output_table="yd_fp_investor_staging.data_engineering_sandbox.variant_coltype_test_deliverable_deliverable2",
    query_template=120,
    slug="variant_column_test_deliverable2", 
    allow_major_version=True,
    allow_minor_version=True,
)

In [0]:
model_version = deliverable['id']

In [0]:
materialization = materialize_deliverable(model_version)

In [0]:
materialization_id = materialization['id']

In [0]:
release_materialization(materialization_id)

fetching latest table version, a release should have been completed before fetching the latest table

In [0]:
while True:
    response = get(f"api/v1/data_model/fp_materialization/{materialization_id}")
    print(response)
    
    last_fp_release = response.json()["last_fp_release"]
    if last_fp_release:

        if last_fp_release['airflow_status'] == "success":
            print(f"last FP Release ready")
            break
        elif last_fp_release['airflow_status']  == "failed":
            print("FP Release failed!")
            break
        
        print("Release still running. Waiting 45 seconds...")
        time.sleep(45)

    else:
        print("No releases found. Waiting 45 seconds...")
        time.sleep(45)

In [0]:
latest_table_name = last_fp_release['view_details']['table_name']
latest_catalog = last_fp_release['view_details']['catalog_name']
latest_database = last_fp_release['view_details']['database_name']

latest_table = f"{latest_catalog}.{latest_database}.{latest_table_name}"
print(f"latest table: {latest_table}")

Add QA on latest table to double check new view source

In [0]:
%sql
--HERE CREATE OR REPLACE VIEW AFTER QA
select *
from yd_fp_investor_staging.data_engineering_sandbox.variant_coltype_test_deliverable_deliverable__dmv__001 limit 40

In [0]:
%sql
describe table yd_fp_investor_staging.data_engineering_sandbox.variant_coltype_test_deliverable_deliverable__dmv__001